-----------------------------------------------------------------------------------------------------------------------
# **Aplicaciones de PLN y Paquetes de Python**
-----------------------------------------------------------------------------------------------------------------------

------------------------------
## **Contexto**
------------------------------

En el contexto de este ejercicio, se plantea el escenario de una empresa de distribución de alimentos con cobertura nacional y un volumen diario de miles de pedidos. La compañía cuenta con un archivo histórico que recoge las solicitudes de comida realizadas por los clientes a través del chat de su página web durante los últimos meses. Dado que la gestión eficiente del inventario es crucial para evitar la escasez de platos preparados, se requiere un sistema capaz de analizar en tiempo real las solicitudes de los usuarios, identificando tanto los platos demandados como los ingredientes asociados. Se ha determinado que la falta de disponibilidad de un plato conlleva, en promedio, una disminución del 7% en las ventas semanales, atribuida al abandono de la plataforma por parte de los clientes. Por ello, la implementación de un mecanismo automatizado para la estimación y predicción de la demanda resulta estratégica para minimizar las pérdidas y asegurar la continuidad operativa

------------------------------
## **Objetivo**
------------------------------

El objetivo es programar una función que reciba como input un texto de usuario y devuelva los fragmentos de texto que hagan referencia a las comidas y cantidades que ha solicitado.

Por ejemplo: “quiero 3 bocadillos de anchoas y 2 pizzas” →
[
{comida:'bocadillo', ingrediente:'anchoas', cantidad:3},
{comida:'pizza', ingrediente:'null', cantidad:2}
]

## **Desarrollo del ejercicio**

Se define el siguiente corpus con el que se entrenará al tomador de pedidos:

In [ ]:
pedido = '''
me gustaria una hamburguesa con cocacola.
quisiera tres sandwiches y dos cocacolas.
me gustaria pedir una pizza.
quisiera una pizza y tres hamburguesas.
envienme patatas y zumo.
me gustaria tomar dos pizzas de atun y un bocadillo.
quisiera tres hamburguesas.
quisiera dos pizzas.
quisiera un sandwich.
quisiera patatas.
quisiera una hamburguesa y un helado
me gustaria un sandwich y una pizza
me apetecen cuatro sandwiches de pavo y una botella de agua.
me apetece un bocadillo de lomo y patatas.
querria dos pizzas.
os queria pedir dos cocacolas.
un sandwich por favor.
queria dos bocadillos de chorizo y un zumo.
quiero un plato de pasta, una empanada y una botella de agua.
quiero tres cocacolas.
querria un sandwich y una pizza.
envienme dos sandwiches y una cocacola.
una pizza por favor.
quiero una hamburguesa por favor.
me gustaria tomar tres magdalenas y un cafe.
quiero que me mandeis cuatro bocadillos de tortilla y un zumo.
envienme tres hamburguesas, patatas y una botella de agua.
envienme dos sandwiches.
me gustaria pedir cuatro pizzas y dos bocadillos.
envienme patatas y zumo.
me gustaria tomar dos pizzas de atun y un bocadillo.
querria tres hamburguesas.
quiero que me mandeis una hamburguesa, dos cocacolas y un helado.
quiero una hamburguesa y diez helados
'''

Se importa el corpus español: cess_esp, y se usa para el entrenamiento del analizador morfológico basado en Modelos de Markov.

In [ ]:
import nltk
from nltk.corpus import cess_esp
from nltk.tag.hmm import HiddenMarkovModelTagger

sentencias_entrenamiento = cess_esp.tagged_sents()
markov_tagger = HiddenMarkovModelTagger.train(sentencias_entrenamiento)

Se usa el modelo entrenado con el corpus importado, para incluir las etiquetas *POS* en el corpus: **pedido**.

In [ ]:
pedidos_2 = nltk.tokenize.sent_tokenize(pedido, language='spanish')

pedidos_tagged = []

for pedido in pedidos_2:
    print("\n")
    pedido_tokens = nltk.word_tokenize(pedido)
    pedido_tagged = markov_tagger.tag(pedido_tokens)
    pedidos_tagged.append(pedido_tagged)
    for (token,tag) in pedido_tagged:
        print(token, tag)



me pp1cs000
gustaria vmip3s0
una di0fs0
hamburguesa ncfs000
con sps00
cocacola np0000l
. Fp


quisiera da0mp0
tres dn0cp0
sandwiches ncmp000
y cc
dos dn0cp0
cocacolas ncmp000
. Fp


me pp1cs000
gustaria vmip3s0
pedir vmn0000
una di0fs0
pizza ncfs000
. Fp


quisiera sps00
una di0fs0
pizza ncfs000
y cc
tres dn0cp0
hamburguesas ncmp000
. Fp


envienme da0fp0
patatas ncfp000
y cc
zumo ncms000
. Fp


me pp1cs000
gustaria vmip3s0
tomar vmn0000
dos dn0cp0
pizzas ncmp000
de sps00
atun np0000l
y cc
un di0ms0
bocadillo ncms000
. Fp


quisiera da0mp0
tres dn0cp0
hamburguesas ncmp000
. Fp


quisiera da0mp0
dos dn0cp0
pizzas ncmp000
. Fp


quisiera sps00
un di0ms0
sandwich ncms000
. Fp


quisiera da0fp0
patatas ncfp000
. Fp


quisiera sps00
una di0fs0
hamburguesa ncfs000
y cc
un di0ms0
helado ncms000
me pp1cs000
gustaria vmip3s0
un di0ms0
sandwich ncms000
y cc
una di0fs0
pizza ncfs000
me pp1cs000
apetecen vmip3s0
cuatro dn0cp0
sandwiches ncmp000
de sps00
pavo np0000l
y cc
una di0fs0
botella ncfs0

Ahora para identificar los productos y cantidades, se crea una gramática sencilla. Se usa para esto *RegexParser*, como sigue:

In [ ]:
gramatica = r"""
  Producto: {<n.*>+ <a.*>? (<sp.*>? <n.*>)?}
  Cantidad: {<Z|di.*>}
  """
regex_parser = nltk.RegexpParser(gramatica)

Se aplica entonces la gramática creada, sobre el texto (*pedido*) con las estiquetas *POS* para identificar los productos y cantidades:

In [ ]:
pedidos_gramatica = []
for pedido_tagged in pedidos_tagged:
    print("")
    pedido_gramatica = regex_parser.parse(pedido_tagged)
    pedidos_gramatica.append(pedido_gramatica)
    print(pedido_gramatica)


(S
  me/pp1cs000
  gustaria/vmip3s0
  (Cantidad una/di0fs0)
  (Producto hamburguesa/ncfs000 con/sps00 cocacola/np0000l)
  ./Fp)

(S
  quisiera/da0mp0
  tres/dn0cp0
  (Producto sandwiches/ncmp000)
  y/cc
  dos/dn0cp0
  (Producto cocacolas/ncmp000)
  ./Fp)

(S
  me/pp1cs000
  gustaria/vmip3s0
  pedir/vmn0000
  (Cantidad una/di0fs0)
  (Producto pizza/ncfs000)
  ./Fp)

(S
  quisiera/sps00
  (Cantidad una/di0fs0)
  (Producto pizza/ncfs000)
  y/cc
  tres/dn0cp0
  (Producto hamburguesas/ncmp000)
  ./Fp)

(S
  envienme/da0fp0
  (Producto patatas/ncfp000)
  y/cc
  (Producto zumo/ncms000)
  ./Fp)

(S
  me/pp1cs000
  gustaria/vmip3s0
  tomar/vmn0000
  dos/dn0cp0
  (Producto pizzas/ncmp000 de/sps00 atun/np0000l)
  y/cc
  (Cantidad un/di0ms0)
  (Producto bocadillo/ncms000)
  ./Fp)

(S quisiera/da0mp0 tres/dn0cp0 (Producto hamburguesas/ncmp000) ./Fp)

(S quisiera/da0mp0 dos/dn0cp0 (Producto pizzas/ncmp000) ./Fp)

(S
  quisiera/sps00
  (Cantidad un/di0ms0)
  (Producto sandwich/ncms000)
  ./Fp)

(S q

La gramática creada no identifica correctamente las cantidades: dos o tres. Se incluye este particular, en la gramática, para solucionarlo:

In [ ]:
gramatica = r"""
  Producto: {<n.*>+ <a.*>? (<sp.*>? <n.*>)?}
  Cantidad: {<Z|di.*>}
  Cantidad: {<Z|dn.*>}
  """
regex_parser = nltk.RegexpParser(gramatica)

In [ ]:
pedidos_gramatica = []
for pedido_tagged in pedidos_tagged:
    print("")
    pedido_gramatica = regex_parser.parse(pedido_tagged)
    pedidos_gramatica.append(pedido_gramatica)
    print(pedido_gramatica)


(S
  me/pp1cs000
  gustaria/vmip3s0
  (Cantidad una/di0fs0)
  (Producto hamburguesa/ncfs000 con/sps00 cocacola/np0000l)
  ./Fp)

(S
  quisiera/da0mp0
  (Cantidad tres/dn0cp0)
  (Producto sandwiches/ncmp000)
  y/cc
  (Cantidad dos/dn0cp0)
  (Producto cocacolas/ncmp000)
  ./Fp)

(S
  me/pp1cs000
  gustaria/vmip3s0
  pedir/vmn0000
  (Cantidad una/di0fs0)
  (Producto pizza/ncfs000)
  ./Fp)

(S
  quisiera/sps00
  (Cantidad una/di0fs0)
  (Producto pizza/ncfs000)
  y/cc
  (Cantidad tres/dn0cp0)
  (Producto hamburguesas/ncmp000)
  ./Fp)

(S
  envienme/da0fp0
  (Producto patatas/ncfp000)
  y/cc
  (Producto zumo/ncms000)
  ./Fp)

(S
  me/pp1cs000
  gustaria/vmip3s0
  tomar/vmn0000
  (Cantidad dos/dn0cp0)
  (Producto pizzas/ncmp000 de/sps00 atun/np0000l)
  y/cc
  (Cantidad un/di0ms0)
  (Producto bocadillo/ncms000)
  ./Fp)

(S
  quisiera/da0mp0
  (Cantidad tres/dn0cp0)
  (Producto hamburguesas/ncmp000)
  ./Fp)

(S
  quisiera/da0mp0
  (Cantidad dos/dn0cp0)
  (Producto pizzas/ncmp000)
  ./Fp)

(S
 

Se revisa el resultado y la identificación de productos - cantidades es correcto. Se procede a generar las etiquetas *IOB*.

In [ ]:
for pedido_gramatica in pedidos_gramatica:
    for palabra in pedido_gramatica:
        if isinstance(palabra, nltk.tree.Tree):
            primera_palabra = True
            for e in palabra:
                token = e[0]
                etiqueta_pos = e[1]
                etiqueta_iob = 'I'
                if primera_palabra:
                    etiqueta_iob = 'B'
                    primera_palabra = False
                print(token, etiqueta_pos, palabra.label() + '-' + etiqueta_iob)
        else:
            token = palabra[0]
            etiqueta_pos = palabra[1]
            print(token, etiqueta_pos, 'O')
    print("\n")

me pp1cs000 O
gustaria vmip3s0 O
una di0fs0 Cantidad-B
hamburguesa ncfs000 Producto-B
con sps00 Producto-I
cocacola np0000l Producto-I
. Fp O


quisiera da0mp0 O
tres dn0cp0 Cantidad-B
sandwiches ncmp000 Producto-B
y cc O
dos dn0cp0 Cantidad-B
cocacolas ncmp000 Producto-B
. Fp O


me pp1cs000 O
gustaria vmip3s0 O
pedir vmn0000 O
una di0fs0 Cantidad-B
pizza ncfs000 Producto-B
. Fp O


quisiera sps00 O
una di0fs0 Cantidad-B
pizza ncfs000 Producto-B
y cc O
tres dn0cp0 Cantidad-B
hamburguesas ncmp000 Producto-B
. Fp O


envienme da0fp0 O
patatas ncfp000 Producto-B
y cc O
zumo ncms000 Producto-B
. Fp O


me pp1cs000 O
gustaria vmip3s0 O
tomar vmn0000 O
dos dn0cp0 Cantidad-B
pizzas ncmp000 Producto-B
de sps00 Producto-I
atun np0000l Producto-I
y cc O
un di0ms0 Cantidad-B
bocadillo ncms000 Producto-B
. Fp O


quisiera da0mp0 O
tres dn0cp0 Cantidad-B
hamburguesas ncmp000 Producto-B
. Fp O


quisiera da0mp0 O
dos dn0cp0 Cantidad-B
pizzas ncmp000 Producto-B
. Fp O


quisiera sps00 O
un di0ms0 Ca

Los resultados obtenidos son relativamente buenos, existen oportunidades de mejora por ejemplo en palabras como por favor, que actualmente la gramática las clasifica como un producto, al ser palabras tan comunes se debería corregit manualmente este particular. Sin embargo, el objetivo de la práctica es enfocarse en los conceptos y no en la efectividad de la proyección, por lo que no se realizará esta corrección.

Se procede entonces a definir la clase *UnigramChunker*, como sigue:

In [ ]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

Se entrena un modelo basado en *Unigramas*. Para este objetivo se usa el corpus *pedidos_gramatica*:

In [ ]:
modelo_unigramas = UnigramChunker(pedidos_gramatica)

Para verificar que el modelo esté construido correctamente se prueba un pedido no incluido en el corpus de entrenamiento:

In [ ]:
pedido_prueba = "Quisiera una hamburguesa, una pizza de peperoni y dos zumos de naranja"
tokens = nltk.word_tokenize(pedido_prueba)
tokens_etiquetados = markov_tagger.tag(tokens)
pedido_etiquetado = modelo_unigramas.parse(tokens_etiquetados)

for palabra in pedido_etiquetado:
    print (palabra)

(Producto Quisiera/sps00)
(Cantidad una/di0fs0)
(Producto hamburguesa/ncfs000)
(',', 'Fc')
(Cantidad una/di0fs0)
(Producto pizza/ncfs000 de/sps00 peperoni/np0000l)
('y', 'cc')
(Cantidad dos/dn0cp0)
(Producto zumos/ncmp000 de/sps00)
(Producto naranja/ncfs000)


El modelo identifica la palabra *'Quisiera'* como *Producto*. Por otro lado, tiene problemas en identificar un producto que contiene más de un sustantivo. Se prueba entonces un modelo con *Bigramas* para solucionar este problema. Se define entonces la clase BigramChunker como sigue:

In [ ]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

modelo_bigramas = BigramChunker(pedidos_gramatica)

Se prueba con este modelo la fresa anterior, obteniendo los siguinetes resultados:

In [ ]:
tokens = nltk.word_tokenize(pedido_prueba)
tokens_etiquetados = markov_tagger.tag(tokens)
pedido_etiquetado = modelo_bigramas.parse(tokens_etiquetados)

for palabra in pedido_etiquetado:
    print (palabra)

('Quisiera', 'sps00')
(Cantidad una/di0fs0)
(Producto hamburguesa/ncfs000)
(',', 'Fc')
(Cantidad una/di0fs0)
(Producto pizza/ncfs000 de/sps00 peperoni/np0000l)
('y', 'cc')
(Cantidad dos/dn0cp0)
(Producto zumos/ncmp000 de/sps00 naranja/ncfs000)


La clasificación que ha dado el modelo es correcta. Se procede entonces a probar un modelo *Naive Bayes*, con el objetivo de mejorar la precisión en la clasificación de produtos y cantidades:

In [ ]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

Además de las variables *pos*, *word* y *prevpos* que se incluye en la función original *npchunk_features* (http://www.nltk.org/book/ch07.html), se pueden incluir otras variables que son de utilidad para el modelo, en este caso se usará: el número de letras de cada palabra.

In [ ]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "word": word, "prevpos": prevpos, 'length': len(word)}

Una vez que se han realizado todas las definiciones necesarias para el modelo *Naive Bayes*, se lo entrena y prueba sobre la frase que se ha usado para los modelos de unigramas y bigramas respectivamente:

In [ ]:
modelo_NaiveBayes = ConsecutiveNPChunker(pedidos_gramatica)

tokens = nltk.word_tokenize(pedido_prueba)
tokens_etiquetados = markov_tagger.tag(tokens)
pedido_etiquetado = modelo_NaiveBayes.parse(tokens_etiquetados)

for palabra in pedido_etiquetado:
    print (palabra)

('Quisiera', 'sps00')
(Cantidad una/di0fs0)
(Producto hamburguesa/ncfs000)
(',', 'Fc')
(Cantidad una/di0fs0)
(Producto pizza/ncfs000 de/sps00 peperoni/np0000l)
('y', 'cc')
(Cantidad dos/dn0cp0)
(Producto zumos/ncmp000 de/sps00 naranja/ncfs000)


Los resultados son correctos, al igual que se observó con Bigramas. Ahora, ya que no se dispone de un conjunto de frases bien clasificadas, para poder probar la precisión de cada modelo. Se trabajará sobre muestras aleatorias de entrenamiento y test. De modo que se pueda seleccionar al mejor modelo, que posteriormente se usará para culminar el ejercicio.

In [ ]:
import random
random.seed(256)

accuracy_unigram = []
accuracy_bigram = []
accuracy_NaiveBayes = []

precisiones_unigram = []
precisiones_bigram = []
precisiones_NaiveBayes = []

recall_unigram = []
recall_bigram = []
recall_NaiveBayes = []

F_unigram = []
F_bigram = []
F_NaiveBayes = []

# Se usan 20 iteraciones para calcular los indicadores de rendimiento:
for iteraciones in range(20):

    # Genración aletoria de los índices para los conjuntos de entrenamiento y test
    indices_test = sorted(random.sample(range(len(pedidos_gramatica)), 8))
    indices_entrenamiento = []
    for i in range(len(pedidos_gramatica)):
        if i not in indices_test:
            indices_entrenamiento.append(i)

    # Generación de conjuntos de entrenamiento y test
    test = [ pedidos_gramatica[i] for i in indices_test ]
    entrenamiento = [ pedidos_gramatica[i] for i in indices_entrenamiento ]

    # Entrenamiento
    unigramas = UnigramChunker(entrenamiento)
    bigramas = BigramChunker(entrenamiento)
    NaiveBayes = ConsecutiveNPChunker(entrenamiento)

    # Calculo de indicadores sobre Test
    accuracy_unigram.append(unigramas.evaluate(test).accuracy())
    accuracy_bigram.append(bigramas.evaluate(test).accuracy())
    accuracy_NaiveBayes.append(NaiveBayes.evaluate(test).accuracy())

    precisiones_unigram.append(unigramas.evaluate(test).precision())
    precisiones_bigram.append(bigramas.evaluate(test).precision())
    precisiones_NaiveBayes.append(NaiveBayes.evaluate(test).precision())

    recall_unigram.append(unigramas.evaluate(test).recall())
    recall_bigram.append(bigramas.evaluate(test).recall())
    recall_NaiveBayes.append(NaiveBayes.evaluate(test).recall())

    F_unigram.append(unigramas.evaluate(test).f_measure())
    F_bigram.append(bigramas.evaluate(test).f_measure())
    F_NaiveBayes.append(NaiveBayes.evaluate(test).f_measure())

print("Modelo Unigramas:")
print(" - IOB Accuracy: " + str(sum(accuracy_unigram)/20))
print(" - Precision: " + str(sum(precisiones_unigram)/20))
print(" - Recall: " + str(sum(recall_unigram)/20))
print(" - F-measure: " + str(sum(F_unigram)/20))
print("\n")
print("Modelo Bigramas:")
print(" - IOB Accuracy: " + str(sum(accuracy_bigram)/20))
print(" - Precision: " + str(sum(precisiones_bigram)/20))
print(" - Recall: " + str(sum(recall_bigram)/20))
print(" - F-measure: " + str(sum(F_bigram)/20))
print("\n")
print("Modelo Naive Bayes:")
print(" - IOB Accuracy: " + str(sum(accuracy_NaiveBayes)/20))
print(" - Precision: " + str(sum(precisiones_NaiveBayes)/20))
print(" - Recall: " + str(sum(recall_NaiveBayes)/20))
print(" - F-measure: " + str(sum(F_NaiveBayes)/20))

Modelo Unigramas:
 - IOB Accuracy: 0.9406404703675919
 - Precision: 0.8263555577480259
 - Recall: 0.9255508454949574
 - F-measure: 0.8723397702451676


Modelo Bigramas:
 - IOB Accuracy: 0.8649005028483738
 - Precision: 1.0
 - Recall: 0.7983025318139402
 - F-measure: 0.8807597406436936


Modelo Naive Bayes:
 - IOB Accuracy: 0.9884820694032028
 - Precision: 0.9747239356316436
 - Recall: 1.0
 - F-measure: 0.9869900435953927


El modelo basado en Bigramas es que el que mayor precisión tiene en clasificar entre productos y cantidades. Por lo que será el modelo que se use, para etiquetar los pedidos de un cliente.

Se crea entonces una función que devuelva un arreglo que permita indentificar el producto y la cantidad requerida.

In [ ]:
def pedir(pedido):
    tokens = nltk.word_tokenize(pedido)
    tokens_etiquetados = markov_tagger.tag(tokens)
    pedido_etiquetado = modelo_bigramas.parse(tokens_etiquetados)
    salida = []
    cantidad = ""
    comida = ""

    for palabra in pedido_etiquetado:
        if isinstance(palabra, nltk.tree.Tree):
            etiqueta = palabra.label()
            valor = ""
            for e in palabra:
                valor = valor + e[0] + " "

            if etiqueta == "Cantidad":
                cantidad = valor[:]
            else:
                if etiqueta == "Producto":
                    comida = valor[:]

            if comida != "":

                if cantidad == "":
                    cantidad = "un"
                salida.append({"Cantidad": cantidad, "Producto": comida})
                cantidad = "un"
                comida = ""
    return(salida)

Para evaluar que la función esté correcta, se prueba un par de frases como sigue:

In [ ]:
pedir("quiero cuatro pizzas")

[{'Cantidad': 'cuatro ', 'Producto': 'pizzas '}]

In [ ]:
pedir("quiero un vino y un zumo de naranja")

[{'Cantidad': 'un ', 'Producto': 'vino '},
 {'Cantidad': 'un ', 'Producto': 'zumo de naranja '}]

Finalmente, para receptar los pedidos de los clientes se crea un programa que permita interactuar con el cliente, haciendo la idea de tener un chatboot:

In [ ]:
terminar = False
while not terminar:
    entrada = input("Hola, escribe tu pedido y pulsa enter (teclea 'fin' si deseas terminar): ")
    if entrada == "fin":
        terminar = True
    else:
        print(pedir(entrada))
        print("\n")

Hola, escribe tu pedido y pulsa enter (teclea 'fin' si deseas terminar): quiero un vino y dos zumos de naranja
[{'Cantidad': 'un ', 'Producto': 'vino '}, {'Cantidad': 'dos ', 'Producto': 'zumos de naranja '}]


Hola, escribe tu pedido y pulsa enter (teclea 'fin' si deseas terminar): quiero dos hamburguesas
[{'Cantidad': 'dos ', 'Producto': 'hamburguesas '}]


Hola, escribe tu pedido y pulsa enter (teclea 'fin' si deseas terminar): fin
